# Embedding models into an application

We will now see how to embed a model into an application that can learn and predict in real time.

Let's start by loading the **pickle** object we created at the end of chapter 8

In [1]:
import pickle 
import os
import re

In [2]:
os.chdir('movieclassifier')

from vectorizer import vect

clf = pickle.load(open(os.path.join('pkl_objects', 'classifier.pkl'), 'rb'))

Now we can use the loaded classifier to preprocess samples and make predictions

In [6]:
import numpy as np

label = {0:'negative', 1:'positive'}
example = ['I love this movie']
x = vect.transform(example)
print('Predicton: %s\nProbability: %.2f' % (label[clf.predict(x)[0]], np.max(clf.predict_proba(x))*100))

Predicton: positive
Probability: 79.25


## Setting up SQlite for storage

With [**SQlite**](http://www.sqlite.org/) we can safely store data for simple apps without running a separate server for a database. In Python there is [**sqlite3**](https://docs.python.org/3.4/library/sqlite3.html) a core package for dealing with such databases.

By setting a database we can store feedback about the app and collecting new samples. Let's start a db inside the *movieclassifier* directory and store two example movie reviews.

In [7]:
import sqlite3
import os

In [9]:
conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()
c.execute('CREATE TABLE review_db (review TEXT, sentiment INTEGER, date TEXT)')

example1 = 'I love this movie'
c.execute('INSERT INTO review_db (review, sentiment, date) VALUES (?, ?, DATETIME("now"))', 
          (example1, 1))

example2 = 'I disliked this movie'
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES (?, ?, DATETIME('now))",
          (example2, 0))

conn.commit()
conn.close()

OperationalError: table review_db already exists

In [14]:
conn = sqlite3.connect("reviews.sqlite")
c = conn.cursor()
c.execute("SELECT * FROM review_db WHERE date BETWEEN '2017-01-01 00:00:00' AND DATETIME('now')")
results = c.fetchall()
conn.close()
print(results)

[('I love this movie', 1, '2017-02-24 21:51:16'), ('I disliked this movie', 0, '2017-02-24 21:51:16')]


## Developing a web application with Flask

